$\textbf{x}\in\Omega$

$$\nabla^2\textbf{u} = \textbf{f}(\textbf{x})$$

$\textbf{x}\in\partial\Omega_{\text{D}}$ Dirichlet boundary condition

$$\textbf{u} = \textbf{u}_{\text{D}}$$

$\textbf{x}\in\partial\Omega_{\text{N}}=\partial\Omega/\partial\Omega_{\text{D}}$ Neumann condition

$$\textbf{n}\cdot\nabla\textbf{u} = \textbf{u}_{\text{N}}$$

variational formulation

$$-\int_\Omega\text{d}\Omega~\nabla\textbf{v}\cdot\nabla\textbf{u} + \textbf{v}\cdot\textbf{f} + \int_{\partial\Omega}\text{d}\Gamma~\textbf{v}\cdot(\textbf{n}\cdot\nabla\textbf{u})=0$$



In [ ]:
from ufl.core.expr import Expr
from ufl import Form, inner, grad, nabla_grad, dx, TestFunction, TrialFunction
from dolfinx.fem import FunctionSpace

from lucifex.mesh import rectangle_mesh, mesh_boundary
from lucifex.fem import LUCiFExFunction as Function, LUCiFExConstant as Constant
from lucifex.solver import bvp_solver, BoundaryConditions
from lucifex.viz import plot_colormap, plot_line
from lucifex.utils import cross_section, fem_function_components


def poisson_vector(
    u: Function,
    f: Function | Constant | Expr,
) -> tuple[Form, Form]:
    v = TestFunction(u.function_space)
    u_trial = TrialFunction(u.function_space)
    F_lhs = -inner(grad(v), grad(u_trial)) * dx
    F_rhs = -inner(v, f) * dx
    return F_lhs, F_rhs

### $d=2~,~\textbf{u}\in\mathbb{R}^2$

$$\Omega = [0, L_x] \times [0, L_y]$$

$$\textbf{f}(x,y)=(-y, x)$$

$$\textbf{u}(x=0,y)=(0, 0)$$

$$\textbf{u}(x=L_x,y)=(1, -1)$$

$$\textbf{u}(x,y=0)=(x, 1)$$

$$\textbf{u}(x,y=L_y)=(1, 2)$$

In [ ]:
Lx = 2.0
Ly = 1.0
mesh = rectangle_mesh(Lx, Ly, 20, 10)
boundary = mesh_boundary(
    mesh, 
    {
        "left": lambda x: x[0],
        "right": lambda x: x[0] - Lx,
        "lower": lambda x: x[1],
        "upper": lambda x: x[1] - Ly,
    },
)
fs = FunctionSpace(mesh, ('P', 1, 2))
f = Function(fs, lambda x: (-x[1], x[0]), name='f')

bcs = BoundaryConditions(
    ('dirichlet', boundary['left'], (0.0, 0.0)),
    ('dirichlet', boundary['right'], Constant(mesh, (1.0, -1.0))),
    ('dirichlet', boundary['lower'], (lambda x: x[0], 1.0), 1),
    ('dirichlet', boundary['upper'], 1.0, 0),
    ('dirichlet', boundary['upper'], 2.0, 1),
)

u = Function(fs, name='u')
u_solver = bvp_solver(poisson_vector, bcs)(u, f)
u_solver.solve()

ux, uy = fem_function_components(('P', 1), u, names=('ux', 'uy'))

In [ ]:
x_axis, ux_x, y_value = cross_section(ux, 'y', 0.5)
plot_line((x_axis, ux_x), x_label='$x$', y_label=f'{u.name}_x(y={y_value:.2f})')

y_axis, ux_y, x_value = cross_section(ux, 'x', 0.5)
plot_line((y_axis, ux_y), x_label='$y$', y_label=f'{u.name}_x(x={y_value:.2f})')

In [ ]:
x_axis, uy_x, y_value = cross_section(uy, 'y', 0.5)
plot_line((x_axis, uy_x), x_label='$x$', y_label=f'{u.name}_y(y={y_value:.2f})')

y_axis, uy_y, x_value = cross_section(uy, 'x', 0.5)
plot_line((y_axis, uy_x), x_label='$y$', y_label=f'{u.name}_y(x={y_value:.2f})')